In [ ]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import pickle
import sys
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from torch import optim
import torch.nn as nn
#from ipdb import set_trace
from datetime import datetime
import time
import copy
from tqdm import tqdm
from ipdb import set_trace
import argparse
from torch.utils.data import Dataset
from matplotlib import image
from matplotlib import pyplot
from torchvision import transforms
from PIL import Image
import imgaug as ia
import imgaug.augmenters as iaa



# Ignore warnings
# import warnings
# warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

# Util functions

In [ ]:
def onehot(vals, possible_vals):
    if not isinstance(possible_vals, list): raise TypeError("provide possible_vals as a list")
    enc_vals = np.zeros([len(vals), len(possible_vals)])
    for i, value in enumerate(vals):
        if isinstance(possible_vals[0], float):
            enc = np.where(abs(possible_vals-value)<1e-3)
        else:
            enc = np.where(possible_vals==value)
        enc_vals[i,enc] = 1
    return enc_vals

class Rescale(object):
    def __init__(self, scalar):
        self.scalar = scalar

    def __call__(self, im):
        w, h = [int(s*self.scalar) for s in im.size]
        return transforms.Resize((h, w))(im)

class Crop(object):
    def __init__(self, box):
        assert len(box) == 4
        self.box = box

    def __call__(self, im):
        return im.crop(self.box)

class Augment(object):
    def __init__(self, seq):
        self.seq = seq

    def __call__(self, im):
        return Image.fromarray(self.seq.augment_images([np.array(im)])[0])

def load_checkpoint(filename) :
    #print(filename)
    checkpoint = torch.load(filename)
    model_state_dict = checkpoint['model_state_dict']
    optimizer_state_dict = checkpoint['optimizer_state_dict']
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    return [model_state_dict, optimizer_state_dict, epoch, loss]

def save_checkpoint(model, optimizer, epoch, loss, filename):
    print("Saving Checkpoint....")
    model_state_dict = model.state_dict()
    optimizer_state_dict = optimizer.state_dict()
    torch.save({
            'epoch': epoch,
            'model_state_dict': model_state_dict,
            'optimizer_state_dict': optimizer_state_dict,
            'loss': loss
            }, filename)
    print("Saving Checkpoint Done")

def display_image(image) : 
    plt.show(image)
    
def load_checkpoint_weights(checkpoint, model) :
    print("checkpoint name : ", checkpoint)
    _model_state_dict, _optimizer_state_dict, _epoch, _loss = load_checkpoint(checkpoint)
    model.load_state_dict(_model_state_dict)

def resume_model(checkpoint, model, optimizer) : 
    print("checkpoint name : ", checkpoint)
    _model_state_dict, _optimizer_state_dict, _epoch, _loss = load_checkpoint(checkpoint)
    #print(_model_state_dict)
    model.load_state_dict(_model_state_dict)
    optimizer.load_state_dict(_optimizer_state_dict)
    print("resume model succesful")

# Model functions

In [ ]:
def train_model(model, dataloaders, dataset_sizes, criterion, optimizer, scheduler, num_epochs=30):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_loss = 0.0
    epoch_acc = 0.0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for i, data in enumerate(dataloaders[phase]) :
                inputs_old = data[0]
                #print("input before convertion", inputs_old)
                labels = data[1]
                inputs, labels = convert_inputs(inputs_old ,labels)
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    #_, preds = torch.max(outputs, 1)
                    preds = outputs
                    #print(outputs.size(), labels.size())
                    #input("Enter")
                    loss = criterion(outputs, labels)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        if(i%100 == 0) :
                            print("preds shape : ", preds.size(), " labels shape : ", labels.size())
                            print("preds : ", preds[0:5].T.data, "\nlabels : ", labels.data[0:5].T.data)
                            print("loss :", loss.item())

                # statistics
                running_loss += loss.item() * inputs.size(0)
                #running_corrects += torch.sum(preds == labels.data)

                if(phase == 'val') :
                    if(i%10==0) :
                        print("preds : ", preds[0:5].T.data, "\nlabels : ", labels.data[0:5].T.data)
                        print("loss :", loss.item())
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            #epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            #if phase == 'val' and epoch_acc > best_acc:
            if phase == 'val' and epoch_loss < best_loss:
                #best_acc = epoch_acc
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'train'  :
                checkpoint_name = 'checkpoints/model_' + str(epoch) + '.tar'
                save_checkpoint(model, optimizer, epoch, epoch_loss, checkpoint_name)
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

# Dataset

In [ ]:
def get_augmentations():
    # applies the given augmenter in 50% of all cases,
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)

    # Define our sequence of augmentation steps that will be applied to every image
    seq = iaa.Sequential([
            # execute 0 to 5 of the following (less important) augmenters per image
            iaa.SomeOf((0, 5),
                [
                    iaa.OneOf([
                        iaa.GaussianBlur((0, 3.0)),
                        iaa.AverageBlur(k=(2, 7)),
                        iaa.MedianBlur(k=(3, 11)),
                    ]),
                    iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)),
                    iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)),
                    # search either for all edges or for directed edges,
                    # blend the result with the original image using a blobby mask
                    iaa.SimplexNoiseAlpha(iaa.OneOf([
                        iaa.EdgeDetect(alpha=(0.5, 1.0)),
                        iaa.DirectedEdgeDetect(alpha=(0.5, 1.0), direction=(0.0, 1.0)),
                    ])),
                    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
                    iaa.OneOf([
                        iaa.Dropout((0.01, 0.1), per_channel=0.5), # randomly remove up to 10% of the pixels
                        iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05), per_channel=0.2),
                    ]),
                    iaa.Add((-10, 10), per_channel=0.5), # change brightness of images (by -10 to 10 of original value)
                    iaa.AddToHueAndSaturation((-20, 20)), # change hue and saturation
                    # either change the brightness of the whole image (sometimes
                    # per channel) or change the brightness of subareas
                    iaa.OneOf([
                        iaa.Multiply((0.5, 1.5), per_channel=0.5),
                        iaa.FrequencyNoiseAlpha(
                            exponent=(-4, 0),
                            first=iaa.Multiply((0.5, 1.5), per_channel=True),
                            second=iaa.ContrastNormalization((0.5, 2.0))
                        )
                    ]),
                    iaa.ContrastNormalization((0.5, 2.0), per_channel=0.5), # improve or worsen the contrast
                    sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)), # move pixels locally around (with random strengths)
                ],
                random_order=True
            )
        ],
        random_order=True
    )
    return seq
class Augment(object):
    def __init__(self, seq):
        self.seq = seq

    def __call__(self, im):
        return Image.fromarray(self.seq.augment_images([np.array(im)])[0])

class CARLA_Dataset(Dataset):
    def __init__(self, t, pickle_file_path_l, image_dir_path, pickle_file_path_r):
        #initializing transforms
        assert t in ['train', 'val']
        self.transform = get_data_transforms(t)
#         selabelslf.labels = {}    
        ###############LEFT CAMERA##############
        #reading the title of all images to access the pickle file
        self.image_path = image_dir_path
        self.image_list = os.listdir(image_dir_path)
        self.file_name = []
        for file in self.image_list:
            self.file_name.append(os.path.splitext(file)[0])
            
        #initialize the labels 
        #read pickle file
        self.pickle_list_l = os.listdir(pickle_file_path_l)
        self.pickled_data_l = {}
        for file in self.pickle_list_l:
            f = open((pickle_file_path_l + file), 'rb')  
            self.pickled_data_l.update(pickle.load(f))
            f.close() 
        
        ###############RIGHT CAMERA##############
        #read pickle file
        self.pickle_list_r = os.listdir(pickle_file_path_r)
        self.pickled_data_r = {}
        for file in self.pickle_list_r:
            f = open((pickle_file_path_r + file), 'rb')  
            self.pickled_data_r.update(pickle.load(f))
            f.close() 
        f.close() 

    def __len__(self):
        return len(self.file_name)

    def __getitem__(self, idx):
        frames      = []
        inputs = {}  
        labels = {}
        #####################LEFT CAMERA################
        #reading PIL
#         self.raw_image = Image.open(os.path.join(self.image_path, self.image_list[idx])).convert('RGB')
        raw_image = Image.open(os.path.join(self.image_path, self.image_list[idx])).convert('RGB')
        #pyplot.imshow(self.raw_image_l)
        #pyplot.show()
        
        #reading file name to access the pickle file
        current_fname = self.file_name[idx]
#         print(current_fname)
        chk_camera = current_fname[-3:]

        if chk_camera == '_rt' :
            current_fname_r = current_fname[:-3]
            label_dict = self.pickled_data_r[current_fname_r] 
        else :
            label_dict = self.pickled_data_l[current_fname] 
        label_values = list(label_dict.values())
#         print(label_values)
        #transforming regression labels
        labels['front_vehicle'] = torch.Tensor(np.array(float(label_dict['front_vehicle'])))
        labels['centre_dist'] = torch.Tensor(np.array(float(label_dict['centre_dist'])))
        labels['pedestrian_distance'] = torch.Tensor(np.array(float(label_dict['pedestrian_distance'])))
        
#         print(labels['front_vehicle'])
        #transforming classification labels
        #self.labels_l['traffic_light'] = torch.Tensor(onehot(np.array(label_dict['traffic_light']), [False, 'Green', True, 'Red']))
        #transforming raw PIL image
        im = self.transform(raw_image)
        
        inputs ['sequence'] = im
#         inputs ['raw_image'] = self.raw_image
        #print(inputs ['sequence'].dim())
        return inputs, labels

#dataset support functions
def get_data_transforms(t='train'):
    data_transforms = {
        'train': transforms.Compose([
#             Augment(get_augmentations()),
            Crop((0,120,800,480)),
            Rescale(0.4),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]),
        'val': transforms.Compose([
            Crop((0,120,800,480)),
            Rescale(0.4),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
    return data_transforms[t]

def get_data(batch_size):
    train_ds = CARLA_Dataset( 'train', pickle_file_path_l='Training_data/Left_pickle_file/',
                                    image_dir_path='Training_data/Camera/', 
                                    pickle_file_path_r='Training_data/Right_pickle_file/')
    val_ds = CARLA_Dataset( 'val', pickle_file_path_l='Training_data/Left_pickle_file/',
                                    image_dir_path='Training_data/Camera/', 
                                    pickle_file_path_r='Training_data/Right_pickle_file/')
#     return (len(train_ds),len(val_ds),DataLoader(train_ds, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=10),
#         DataLoader(val_ds, batch_size=batch_size*2, pin_memory=True, num_workers=10),
#     )
    return (len(train_ds),len(val_ds),DataLoader(train_ds, batch_size=batch_size, shuffle=False, num_workers=8),
        DataLoader(val_ds, batch_size=batch_size*2, num_workers=1),
    )

def convert_inputs(inputs_old, labels):
    inputs            = inputs_old['sequence']
    labels_vdistance  = labels['front_vehicle']
    labels_cdistance  = labels['centre_dist']
    labels_direction  = labels['pedestrian_distance']
#     combined_label    = torch.stack([labels_vdistance, labels_cdistance, labels_direction], dim=1)
#     combined_label    = torch.stack(labels_vdistance, dim=1)
    combined_label = (labels_vdistance/50).unsqueeze(dim=1)
#     print("converted labels : ",combined_label.size())
    return inputs, combined_label

In [ ]:
len_train_ds, len_valid_ds, train_dl, valid_dl = get_data(batch_size=64)
print(len_train_ds)
print(len_valid_ds)

dataloaders   = {'train':train_dl, 'val':valid_dl}
print(dataloaders['train'])
dataset_sizes = {'train':len_train_ds, 'val':len_valid_ds}
print("dataset sizes :", dataset_sizes)

# Data unit test

In [ ]:
# def plot_tensor_image(tensor_im) :
#     plt.imshow(tensor_im.permute(1, 2, 0))
#     plt.show()

# train_dataloader = dataloaders['train']
# data             = next(iter(train_dataloader))
# test_input , test_train_label = data
# convert_inputs(test_input, test_train_label)

# test_train_im = test_input['sequence']
# print(test_train_label)
# print(test_train_im.size())
# for i in range(16) :
#     print(test_train_label['front_vehicle'][i])
#     plot_tensor_image(test_train_im[i])

# Model Definitions

In [ ]:
#model definitions
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ft         = models.resnet50(pretrained=True)
criterion        = nn.MSELoss()
optimizer_ft     = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
num_ftrs         = model_ft.fc.in_features
model_ft.fc      = nn.Linear(num_ftrs, 1)
model_ft         = nn.DataParallel(model_ft)
model_ft         = model_ft.to(device)
load_checkpoint_weights('../python/checkpoints/vehicle_distance/model_19.tar', model_ft)
print("copying model to device....", device)

# Training

In [ ]:
# #train model
print("starting training...")
# #if(args.retrain == True) : 
#  #   resume_model(args.checkpoint, model_ft, optimizer_ft)
train_model(model_ft, dataloaders, dataset_sizes, criterion, optimizer_ft, exp_lr_scheduler, 50)